In [33]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import pickle
from PIL import Image
import numpy as np
import glob
import pandas as pd

In [56]:
train_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/train_images.pickle","rb"))
train_labels = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/train_labels.pickle","rb"))
validation_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/validation_images.pickle","rb"))
validation_labels = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/validation_labels.pickle","rb"))
test_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/test_images.pickle","rb"))

In [57]:
train_labels = train_labels - 1
validation_images, validation_labels = train_images[:100], train_labels[:100]
train_images, train_labels = train_images[100:], train_labels[100:]
test_labels = np.zeros(len(test_images),dtype = "uint8").reshape(-1,1)

In [58]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [59]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

In [60]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
# print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 1063
Validation data size: 100


In [61]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=1, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

In [8]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation='softmax')
])

In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

In [10]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [11]:
model.fit(train_ds,
          epochs=50,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

Epoch 1/50
33/33 [==============================] - 30s 829ms/step - loss: 3.5743 - accuracy: 0.2352 - val_loss: 2.4098 - val_accuracy: 0.2292
Epoch 2/50
33/33 [==============================] - 29s 824ms/step - loss: 2.5947 - accuracy: 0.3298 - val_loss: 2.1023 - val_accuracy: 0.2083
Epoch 3/50
33/33 [==============================] - 27s 773ms/step - loss: 2.2512 - accuracy: 0.3663 - val_loss: 1.5795 - val_accuracy: 0.3125
Epoch 4/50
33/33 [==============================] - 28s 804ms/step - loss: 1.8686 - accuracy: 0.4191 - val_loss: 1.5417 - val_accuracy: 0.3750
Epoch 5/50
33/33 [==============================] - 29s 819ms/step - loss: 1.5984 - accuracy: 0.5051 - val_loss: 1.2378 - val_accuracy: 0.5208
Epoch 6/50
33/33 [==============================] - 29s 811ms/step - loss: 1.4763 - accuracy: 0.5120 - val_loss: 1.2636 - val_accuracy: 0.5000
Epoch 7/50
33/33 [==============================] - 29s 818ms/step - loss: 1.4787 - accuracy: 0.5268 - val_loss: 1.1262 - val_accuracy: 0.6042

In [62]:
temp = model.predict(test_ds)

In [69]:
prediction = []
for i in range(0,len(temp)):
    prediction.append(np.argmax(temp[i]))

In [76]:
pred_df = pd.read_csv("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/example.csv")

In [77]:
pred_df['Prediction'] = prediction

In [82]:
pred_df.to_csv("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/prediction.csv",index=False)